In [2]:
import csv
import shutil
import os
import json

### SELECT COVID SAMPLES FROM COUGHVID DATASET 

In [117]:
# If expert labels are available then Quality = poor , no_cough are removed so we obtain 378 samples

In [1]:
count=0
covid_samples_count=0
reduced=0
bad=0

selected_filenames=[]

for file in os.listdir('public_dataset'):
    if file.endswith(".json"):
        count+=1
        
        path=os.path.join('public_dataset',file)
        
        f=open(path,"r")
        data=json.loads(f.read())
        
        if float(data['cough_detected'])>0.9:
            if 'status' in data:
                if(data['status']=='COVID-19'):
                    covid_samples_count+=1

                    if 'expert_labels_1' in data:
                        if(data['expert_labels_1']['quality']=='ok' or data['expert_labels_1']['quality']=='good'):
                            reduced+=1
                            selected_filenames.append(path)
                    elif 'expert_labels_2' in data:
                        if(data['expert_labels_2']['quality']=='ok' or data['expert_labels_2']['quality']=='good'):
                            reduced+=1
                            selected_filenames.append(path)
                    elif 'expert_labels_3' in data:
                        if(data['expert_labels_3']['quality']=='ok' or data['expert_labels_3']['quality']=='good'):
                            reduced+=1
                            selected_filenames.append(path)
                    else:
                        reduced+=1
                        selected_filenames.append(path)

In [2]:
print("Total Number of samples :",count)
print("\nTotal Number of COVID-19 samples with cough_detected score>0.9 :",covid_samples_count)
print("\nExpert labeled COVID-19 samples with quality good/ok OR not expert labelled :",reduced)

Total Number of samples : 20072

Total Number of COVID-19 samples with cough_detected score>0.9 : 441

Expert labeled COVID-19 samples with quality good/ok OR not expert labelled : 378


In [13]:
# Note - ALL FILES WERE CONVERTED TO .wav format first

In [7]:
for i in selected_filenames:
    name=os.path.splitext(i)[0]
    filepath=name+'.wav'
    shutil.copy(filepath,'COUGHVID-COVID-SAMPLES/')

In [8]:
c=0
for f in os.listdir('COUGHVID-COVID-SAMPLES/'):
    c+=1
print(c)

378


In [2]:
#remove the .webm and .ogg files (keep only .wav)
for file in os.listdir('public_dataset'):
    if file.endswith(".webm") or file.endswith(".ogg"):
        os.remove('public_dataset/'+file)

### SELECT HEALTHY SAMPLES FROM COUGHVID DATASET

In [14]:
healthy_samples=0
reduced=0
paths=[]

for file in os.listdir('public_dataset'):
    if file.endswith(".json"):
        
        path=os.path.join('public_dataset',file)
        
        f=open(path,"r")
        data=json.loads(f.read())
        
        if float(data['cough_detected'])>0.9:
            if 'status' in data:
                if(data['status']=='healthy'):
                    healthy_samples+=1

                    if 'expert_labels_1' in data:
                        if((data['expert_labels_1']['quality']=='ok' or data['expert_labels_1']['quality']=='good')):
                            reduced+=1
                            paths.append(path)
                    elif 'expert_labels_2' in data:
                        if((data['expert_labels_2']['quality']=='ok' or data['expert_labels_2']['quality']=='good')):
                            reduced+=1
                            paths.append(path)
                    elif 'expert_labels_3' in data:
                        if((data['expert_labels_3']['quality']=='ok' or data['expert_labels_3']['quality']=='good')):
                            reduced+=1
                            paths.append(path)
                    else:
                        reduced+=1
                        paths.append(path)
                        
    if reduced==1268:
        break

In [18]:
print("Number of healthy cough sound samples considered : ",reduced)

Number of healthy cough sound samples considered :  1268


In [19]:
for i in paths:
    name=os.path.splitext(i)[0]
    filepath=name+'.wav'
    shutil.copy(filepath,'COUGHVID-HEALTHY-SAMPLES/')

### Extract covid cough sound samples from directories not in the kaggle dataset (which is a subset of COSWARA dataset)

In [11]:
path='coswara-new-folders/'

sub_folder=''

with open('dataset-metadata - Sheet1.csv') as csv_file:
    csv_reader=csv.reader(csv_file,delimiter=',')
    
    for row in csv_reader:
        if(row[0]!=sub_folder):
            counter = 1
            sub_folder = row[0]
        
        sub_sub = os.path.join(path,sub_folder,row[1])
        
        if os.path.isdir(sub_sub) and len(os.listdir(sub_sub))!= 0:
            COVID_STATUS=row[3]
            if(COVID_STATUS=='positive_mild' or COVID_STATUS=='positive_asymp' or COVID_STATUS=='positive_moderate'):
                shutil.copy(sub_sub+'/cough-heavy.wav','COSWARA-NEW-COVID-SAMPLES/'+str(sub_folder)+'-h-'+str(counter)+'.wav')
                shutil.copy(sub_sub+'/cough-shallow.wav','COSWARA-NEW-COVID-SAMPLES/'+str(sub_folder)+'-s-'+str(counter)+'.wav')
                counter+=1

### Extract only heavy-cough.wav (remove shallow-cough.wav)

In [8]:
import os
import re

pattern = '[0-9]*-h-[0-9]*.wav$'

for file in os.listdir('COSWARA-NEW-COVID-SAMPLES'):
    if re.match(pattern,file):
        shutil.copy('COSWARA-NEW-COVID-SAMPLES/'+file,'COSWARA-NEW-HEAVY/'+file)